<a href="https://colab.research.google.com/github/rts1988/Duolingo_spaced_repetition/blob/main/Duolingo_q1fulldataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. make big dataset with datadfq1 and possibly coding each lexeme_string as separate. 
2. also try neural network with surface form as sequence in inputs. 

In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 54.8 gigabytes of available RAM



In [2]:
import bz2
import pickle
import _pickle as cPickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

def decompress_pickle(file):
 data = bz2.BZ2File(file, 'rb')
 data = cPickle.load(data)
 return data

def compressed_pickle(title, data):  # do not add extension in filename
 with bz2.BZ2File(title + '.pbz2', 'w') as f: 
  cPickle.dump(data, f)

path_name = '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
datadfq1 = decompress_pickle(path_name+"Unseen_words_training_set.pbz2") 

In [4]:
datadfq1.shape

(7313354, 14)

In [5]:
datadfq1.size/10**6

102.386956

In [6]:
q1lexemes = decompress_pickle(path_name+"q1_lexeme_features.pbz2")

In [7]:
q1lexemes.shape

(12160, 2231)

In [8]:
q1lexemes.size/10**6

27.12896

Estimate of total size of dataset once joined. 

In [9]:
27.12896/12160*7313354 + 102.386956

16418.47973

Considering 100,000 rows at once, joining, combining, and then compressing, saving into a bz2 file. 

In [10]:
datadfq1.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct,student_word,student_lang
7,1.0,1362082032,444407,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,bajo/bajo<pr>,3,3,1,1,"(u:dDwF, 73eecb492ca758ddab5371cf7b5cca32)","(u:dDwF, en)"
15,1.0,1362082130,77,u:dDwF,es,en,73eecb492ca758ddab5371cf7b5cca32,bajo/bajo<pr>,5,5,1,1,"(u:dDwF, 73eecb492ca758ddab5371cf7b5cca32)","(u:dDwF, en)"
20,1.0,1362082274,1634777,u:dDwF,es,en,c84476c460737d9fb905dca3d35ec995,niños/niño<n><m><pl>,9,8,2,2,"(u:dDwF, c84476c460737d9fb905dca3d35ec995)","(u:dDwF, en)"
21,1.0,1362082274,1708699,u:dDwF,es,en,1a913f2ded424985b9c02d0436008511,leo/leer<vblex><pri><p1><sg>,7,5,2,2,"(u:dDwF, 1a913f2ded424985b9c02d0436008511)","(u:dDwF, en)"
22,1.0,1362082274,1708699,u:dDwF,es,en,38b770e66595fea718366523b4f7db3f,libro/libro<n><m><sg>,8,8,4,4,"(u:dDwF, 38b770e66595fea718366523b4f7db3f)","(u:dDwF, en)"


In [11]:
datadfq1 = pd.concat([datadfq1,pd.get_dummies(datadfq1['learning_language'],drop_first=True,prefix='LL')],axis=1)

In [12]:
datadfq1 = pd.concat([datadfq1,pd.get_dummies(datadfq1['ui_language'],drop_first=True,prefix='UIL')],axis=1)

In [13]:
datadfq1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7313354 entries, 7 to 12854218
Data columns (total 20 columns):
 #   Column             Dtype  
---  ------             -----  
 0   p_recall           float64
 1   timestamp          int64  
 2   delta              int64  
 3   user_id            object 
 4   learning_language  object 
 5   ui_language        object 
 6   lexeme_id          object 
 7   lexeme_string      object 
 8   history_seen       int64  
 9   history_correct    int64  
 10  session_seen       int64  
 11  session_correct    int64  
 12  student_word       object 
 13  student_lang       object 
 14  LL_es              uint8  
 15  LL_fr              uint8  
 16  LL_it              uint8  
 17  LL_pt              uint8  
 18  UIL_es             uint8  
 19  UIL_pt             uint8  
dtypes: float64(1), int64(6), object(7), uint8(6)
memory usage: 878.8+ MB


In [11]:
#datadfq1['p_forgot_bin'] = np.where(1 - datadfq1['p_recall'] >0,1,0).astype(int) # we can do a separate 'y' csr later.

In [14]:
datadfq1 = datadfq1.drop(['p_recall','timestamp','user_id','learning_language','ui_language','lexeme_string','session_correct','student_word','student_lang'],axis=1)

In [16]:
datadfq1.shape


(7313354, 11)

In [15]:
datadfq1.head()

,delta,lexeme_id,history_seen,history_correct,session_seen,LL_es,LL_fr,LL_it,LL_pt,UIL_es,UIL_pt
7,444407,73eecb492ca758ddab5371cf7b5cca32,3,3,1,1,0,0,0,0,0
15,77,73eecb492ca758ddab5371cf7b5cca32,5,5,1,1,0,0,0,0,0
20,1634777,c84476c460737d9fb905dca3d35ec995,9,8,2,1,0,0,0,0,0
21,1708699,1a913f2ded424985b9c02d0436008511,7,5,2,1,0,0,0,0,0
22,1708699,38b770e66595fea718366523b4f7db3f,8,8,4,1,0,0,0,0,0


In [17]:
q1lexemes.select_dtypes(include='object')

,lexeme_id,learning_language,lexeme_string,surface_form,lemma_form,pos,modstrings
0,73eecb492ca758ddab5371cf7b5cca32,es,bajo/bajo<pr>,bajo,bajo,pr,[]
1,c84476c460737d9fb905dca3d35ec995,es,niños/niño<n><m><pl>,niños,niño,n,"[m, pl]"
2,1a913f2ded424985b9c02d0436008511,es,leo/leer<vblex><pri><p1><sg>,leo,leer,vblex,"[pri, p1, sg]"
3,38b770e66595fea718366523b4f7db3f,es,libro/libro<n><m><sg>,libro,libro,n,"[m, sg]"
4,4bdb859f599fa07dd5eecdab0acc2d34,es,a/a<pr>,a,a,pr,[]
...,...,...,...,...,...,...,...
12155,822db15e4e056837f908d47692f6969e,pt,ajuda/ajudar<vblex><pri><p3><sg>,ajuda,ajudar,vblex,"[pri, p3, sg]"
12156,aa25ea9dc156919749f444521720432c,fr,conférence/conférence<n><f><sg>,conférence,conférence,n,"[f, sg]"
12157,3db97fbbdd0d4e08b8248b6219a86e4b,fr,liens/lien<n><m><pl>,liens,lien,n,"[m, pl]"
12158,8b53845a67433bd62179e23bcb88b5c3,pt,foi/ir<vblex><ifi><p3><sg>,foi,ir,vblex,"[ifi, p3, sg]"


In [18]:
q1lexemes = q1lexemes.drop(['learning_language','lexeme_string','surface_form','lemma_form','pos','modstrings'],axis=1)

In [19]:
q1lexemes.select_dtypes('object')

,lexeme_id
0,73eecb492ca758ddab5371cf7b5cca32
1,c84476c460737d9fb905dca3d35ec995
2,1a913f2ded424985b9c02d0436008511
3,38b770e66595fea718366523b4f7db3f
4,4bdb859f599fa07dd5eecdab0acc2d34
...,...
12155,822db15e4e056837f908d47692f6969e
12156,aa25ea9dc156919749f444521720432c
12157,3db97fbbdd0d4e08b8248b6219a86e4b
12158,8b53845a67433bd62179e23bcb88b5c3


In [20]:
q1lexemes.size/10**6

27.04384

In [21]:
from scipy.sparse import coo_matrix, vstack

In [22]:
from google.colab import files

In [25]:
count = 0
for i in range(0,datadfq1.shape[0],100000):
  if count%2:
    print(count)
  
  subdata = datadfq1.iloc[i:min(i+100000,datadfq1.shape[0]),:]
  subdata = pd.merge(left = subdata, right = q1lexemes,left_on = 'lexeme_id',right_on = 'lexeme_id',how="left")
  subdata = subdata.drop([col for col in subdata.columns if col.startswith('lexeme_id')],axis=1)
  #print(subdata.columns)
  #print('size: ',subdata.size/10**6)

  mat = coo_matrix(subdata)
  print('size after compression: ',mat.size/10**6)
  if i >1:
    print('stacking: ')
    Xq1 = vstack([Xq1,mat])
    print('saving: ',"Xq1_"+str(count))
    compressed_pickle(path_name+"Xq1_"+str(count), mat)
  else:
    Xq1 = mat
    compressed_pickle(path_name+"Xq1_"+str(count), mat)
  del subdata, mat
  count +=1
print('Saved all, size Xq1: ',Xq1.size/10**6)
compressed_pickle(path_name+"Xq1all", Xq1)

size after compression:  1.589812
1
size after compression:  1.573493
stacking: 
saving:  Xq1_1
size after compression:  1.579614
stacking: 
saving:  Xq1_2
3
size after compression:  1.60606
stacking: 
saving:  Xq1_3
size after compression:  1.590984
stacking: 
saving:  Xq1_4
5
size after compression:  1.588751
stacking: 
saving:  Xq1_5
size after compression:  1.591647
stacking: 
saving:  Xq1_6
7
size after compression:  1.588008
stacking: 
saving:  Xq1_7
size after compression:  1.58394
stacking: 
saving:  Xq1_8
9
size after compression:  1.614355
stacking: 
saving:  Xq1_9
size after compression:  1.594015
stacking: 
saving:  Xq1_10
11
size after compression:  1.591841
stacking: 
saving:  Xq1_11
size after compression:  1.594244
stacking: 
saving:  Xq1_12
13
size after compression:  1.589557
stacking: 
saving:  Xq1_13
size after compression:  1.593106
stacking: 
saving:  Xq1_14
15
size after compression:  1.615642
stacking: 
saving:  Xq1_15
size after compression:  1.596178
stacking: